# 91. 重み共有：CNNのパラメータ効率の秘密

## 学習目標

このノートブックでは、以下を学びます：

1. **重み共有（Weight Sharing）**の定義と仕組み
2. **パラメータ数の比較**：全結合層 vs 畳み込み層
3. **重み共有の数学的表現**
4. **重み共有がもたらす平行移動等変性**

## 目次

1. [重み共有とは](#section1)
2. [パラメータ数の比較](#section2)
3. [数学的定式化](#section3)
4. [重み共有と平行移動等変性](#section4)
5. [実装で確認](#section5)
6. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch, ConnectionPatch
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

import torch
import torch.nn as nn

<a id="section1"></a>
## 1. 重み共有とは

### 定義

**重み共有（Weight Sharing）**とは：

> 同じカーネル（重み）を画像全体にわたってスライドさせ、すべての位置で同じ重みを使用すること

これはCNNの最も重要な特徴の一つです。

In [ ]:
def visualize_weight_sharing():
    """重み共有の概念を可視化"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    
    # 全結合層（重み共有なし）
    ax = axes[0]
    
    # 入力（4x4 = 16ピクセル）
    input_size = 4
    for i in range(input_size):
        for j in range(input_size):
            rect = Rectangle((j*0.12, 0.7 - i*0.12), 0.1, 0.1,
                             linewidth=1, edgecolor='blue', facecolor='lightblue')
            ax.add_patch(rect)
    ax.text(0.24, 0.85, '入力 (4×4)', fontsize=11, ha='center')
    
    # 出力（4x4 = 16ピクセル）
    for i in range(input_size):
        for j in range(input_size):
            rect = Rectangle((0.7 + j*0.12, 0.7 - i*0.12), 0.1, 0.1,
                             linewidth=1, edgecolor='red', facecolor='lightyellow')
            ax.add_patch(rect)
    ax.text(0.94, 0.85, '出力 (4×4)', fontsize=11, ha='center')
    
    # 接続線（全結合 = 全ての入力が全ての出力に接続）
    # 簡略化のため一部のみ表示
    np.random.seed(42)
    colors = plt.cm.tab20(np.linspace(0, 1, 20))
    for k in range(12):
        i1, j1 = np.random.randint(0, 4, 2)
        i2, j2 = np.random.randint(0, 4, 2)
        ax.plot([j1*0.12 + 0.1, 0.7 + j2*0.12], 
               [0.7 - i1*0.12 + 0.05, 0.7 - i2*0.12 + 0.05],
               color=colors[k], alpha=0.3, linewidth=0.5)
    
    ax.text(0.5, 0.15, '全結合層\n各接続に異なる重み\n（重み共有なし）', fontsize=12, 
           ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))
    ax.text(0.5, 0.02, f'パラメータ数: 16 × 16 = 256', fontsize=11, ha='center')
    
    ax.set_xlim(-0.1, 1.2)
    ax.set_ylim(-0.1, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('全結合層：各位置ごとに異なる重み', fontsize=14, fontweight='bold')
    
    # 畳み込み層（重み共有あり）
    ax = axes[1]
    
    # 入力
    for i in range(input_size):
        for j in range(input_size):
            rect = Rectangle((j*0.12, 0.7 - i*0.12), 0.1, 0.1,
                             linewidth=1, edgecolor='blue', facecolor='lightblue')
            ax.add_patch(rect)
    ax.text(0.24, 0.85, '入力 (4×4)', fontsize=11, ha='center')
    
    # カーネル（中央に大きく表示）
    kernel_x, kernel_y = 0.45, 0.55
    kernel_colors = ['#ff6b6b', '#ffd93d', '#6bcb77', '#4d96ff', 
                    '#ff6b6b', '#ffd93d', '#6bcb77', '#4d96ff', '#ff6b6b']
    for i in range(3):
        for j in range(3):
            rect = Rectangle((kernel_x + j*0.08, kernel_y - i*0.08), 0.07, 0.07,
                             linewidth=2, edgecolor='black', 
                             facecolor=kernel_colors[i*3+j], alpha=0.7)
            ax.add_patch(rect)
    ax.text(kernel_x + 0.12, kernel_y + 0.12, '同じカーネル\n(3×3)', fontsize=10, ha='center')
    
    # 出力
    output_size = 2  # (4-3)/1 + 1 = 2
    for i in range(output_size):
        for j in range(output_size):
            rect = Rectangle((0.8 + j*0.12, 0.6 - i*0.12), 0.1, 0.1,
                             linewidth=1, edgecolor='red', facecolor='lightyellow')
            ax.add_patch(rect)
    ax.text(0.92, 0.8, '出力 (2×2)', fontsize=11, ha='center')
    
    # スライドを示す矢印
    ax.annotate('', xy=(0.35, 0.65), xytext=(0.24, 0.65),
               arrowprops=dict(arrowstyle='->', color='green', lw=2))
    ax.annotate('', xy=(0.75, 0.55), xytext=(0.65, 0.55),
               arrowprops=dict(arrowstyle='->', color='green', lw=2))
    
    ax.text(0.5, 0.15, '畳み込み層\n同じカーネルをスライド\n（重み共有）', fontsize=12, 
           ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
    ax.text(0.5, 0.02, f'パラメータ数: 3 × 3 = 9', fontsize=11, ha='center')
    
    ax.set_xlim(-0.1, 1.2)
    ax.set_ylim(-0.1, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('畳み込み層：同じ重みを全位置で共有', fontsize=14, fontweight='bold')
    
    plt.suptitle('重み共有：CNNの核心的アイデア', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_weight_sharing()

### 重み共有の直感的理解

- **全結合層**: 「位置(1,1)から位置(2,2)への接続」と「位置(3,3)から位置(4,4)への接続」は**別の重み**
- **畳み込み層**: どの位置でも**同じカーネル**を使う

これは「画像のどこにあっても、同じパターンは同じ方法で検出すべき」という仮定に基づいています。

<a id="section2"></a>
## 2. パラメータ数の比較

重み共有による最大の利点は、**パラメータ数の劇的な削減**です。

In [ ]:
def compare_parameters():
    """パラメータ数の詳細比較"""
    print("="*70)
    print("パラメータ数の比較：全結合層 vs 畳み込み層")
    print("="*70)
    
    configs = [
        (28, 28, 1, 32, 3),   # MNIST風
        (32, 32, 3, 64, 3),   # CIFAR風
        (224, 224, 3, 64, 7), # ImageNet風
    ]
    
    results = []
    
    for h, w, c_in, c_out, k in configs:
        # 全結合層（入力を同じサイズの出力に変換）
        fc_params = (h * w * c_in) * (h * w * c_out)
        
        # 畳み込み層（same padding想定）
        conv_params = k * k * c_in * c_out + c_out  # +bias
        
        ratio = fc_params / conv_params
        
        results.append((h, w, c_in, c_out, k, fc_params, conv_params, ratio))
        
        print(f"\n入力: {h}×{w}×{c_in}, 出力チャンネル: {c_out}, カーネル: {k}×{k}")
        print(f"  全結合層:  {fc_params:>15,} パラメータ")
        print(f"  畳み込み層: {conv_params:>15,} パラメータ")
        print(f"  削減率:    {ratio:>15,.0f} 倍")
    
    return results

results = compare_parameters()

In [ ]:
def visualize_parameter_comparison():
    """パラメータ数の比較を可視化"""
    fig, ax = plt.subplots(figsize=(12, 6))
    
    labels = ['28×28×1\n(MNIST)', '32×32×3\n(CIFAR)', '224×224×3\n(ImageNet)']
    fc_params = [results[0][5], results[1][5], results[2][5]]
    conv_params = [results[0][6], results[1][6], results[2][6]]
    
    x = np.arange(len(labels))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, fc_params, width, label='全結合層', color='coral', alpha=0.7)
    bars2 = ax.bar(x + width/2, conv_params, width, label='畳み込み層', color='steelblue', alpha=0.7)
    
    ax.set_ylabel('パラメータ数（対数スケール）', fontsize=12)
    ax.set_title('パラメータ数の比較', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend(fontsize=11)
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3, axis='y')
    
    # 削減率を表示
    for i, (fc, conv) in enumerate(zip(fc_params, conv_params)):
        ratio = fc / conv
        ax.annotate(f'{ratio:,.0f}倍削減', 
                   xy=(i, max(fc, conv)), 
                   xytext=(i, max(fc, conv) * 3),
                   ha='center', fontsize=10,
                   arrowprops=dict(arrowstyle='->', color='green'))
    
    plt.tight_layout()
    plt.show()

visualize_parameter_comparison()

### なぜこれほどの削減が可能なのか？

1. **局所的接続**: 各出力は全入力ではなく、局所的な領域のみを見る
2. **重み共有**: 異なる位置でも同じ重みを再利用

この2つの組み合わせにより、ImageNetサイズの画像では**10億倍**以上のパラメータ削減が実現！

<a id="section3"></a>
## 3. 数学的定式化

### 全結合層の出力

$$y_j = \sum_{i} W_{ji} \cdot x_i + b_j$$

ここで $W_{ji}$ は入力 $i$ から出力 $j$ への重みで、**すべて異なる値**です。

### 畳み込み層の出力

$$(x * w)(i, j) = \sum_{m} \sum_{n} w(m, n) \cdot x(i+m, j+n) + b$$

ここで $w(m, n)$ は位置 $(i, j)$ に**依存しない**、共有されたカーネル重みです。

In [ ]:
def demonstrate_weight_sharing_math():
    """重み共有の数学的な違いをデモ"""
    print("="*70)
    print("重み共有の数学的違い")
    print("="*70)
    
    # 5x5入力、3x3カーネル → 3x3出力
    np.random.seed(42)
    x = np.arange(25).reshape(5, 5).astype(float)
    kernel = np.array([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]], dtype=float)
    
    print("\n入力 x (5×5):")
    print(x)
    
    print("\nカーネル w (3×3):")
    print(kernel)
    
    print("\n【重み共有の確認】")
    print("\n位置(0,0)での計算:")
    patch_00 = x[0:3, 0:3]
    output_00 = np.sum(patch_00 * kernel)
    print(f"  パッチ: {patch_00.flatten()}")
    print(f"  カーネル: {kernel.flatten()}")
    print(f"  出力: Σ(パッチ × カーネル) = {output_00}")
    
    print("\n位置(1,1)での計算:")
    patch_11 = x[1:4, 1:4]
    output_11 = np.sum(patch_11 * kernel)
    print(f"  パッチ: {patch_11.flatten()}")
    print(f"  カーネル: {kernel.flatten()}  ← 同じカーネル！")
    print(f"  出力: Σ(パッチ × カーネル) = {output_11}")
    
    print("\n→ 異なる位置でも同じカーネル（重み）を使用")
    print("  これが『重み共有』の本質です")

demonstrate_weight_sharing_math()

In [ ]:
def visualize_shared_vs_unshared():
    """共有重みと非共有重みの違いを可視化"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 共有重み（CNN）
    ax = axes[0]
    
    # カーネル（1つ）
    kernel = np.array([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]])
    
    im = ax.imshow(kernel, cmap='RdBu', vmin=-2, vmax=2)
    for i in range(3):
        for j in range(3):
            ax.text(j, i, f'{kernel[i,j]:+d}', ha='center', va='center', 
                   fontsize=16, fontweight='bold')
    
    ax.set_title('畳み込み層：1つのカーネルを全位置で共有\n（パラメータ数 = 9）', 
                fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.colorbar(im, ax=ax, shrink=0.6)
    
    # 非共有重み（全結合の一部を表示）
    ax = axes[1]
    
    # 4つの異なる「カーネル」を表示（実際にはもっと多い）
    np.random.seed(42)
    
    for idx, (row, col) in enumerate([(0, 0), (0, 1), (1, 0), (1, 1)]):
        # 各位置に異なる重み
        weights = np.random.randn(3, 3)
        
        # サブプロット内の位置
        ax_sub = ax.inset_axes([col*0.5, (1-row)*0.5 - 0.5, 0.45, 0.45])
        im = ax_sub.imshow(weights, cmap='RdBu', vmin=-2, vmax=2)
        ax_sub.set_title(f'位置({row},{col})の重み', fontsize=9)
        ax_sub.set_xticks([])
        ax_sub.set_yticks([])
    
    ax.set_title('全結合層：各位置で異なる重み\n（パラメータ数 = 位置数 × 9）', 
                fontsize=14, fontweight='bold')
    ax.axis('off')
    
    plt.suptitle('重み共有 vs 非共有', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_shared_vs_unshared()

<a id="section4"></a>
## 4. 重み共有と平行移動等変性

重み共有は**平行移動等変性（Translation Equivariance）**をもたらします。

### 平行移動等変性とは

> 入力が空間的にシフトすると、出力も同じだけシフトする性質

$$f(\text{shift}(x)) = \text{shift}(f(x))$$

In [ ]:
def demonstrate_translation_equivariance():
    """平行移動等変性のデモ"""
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    
    # 入力画像1（パターンが左上）
    img1 = np.zeros((8, 8))
    pattern = np.array([[1, 1, 0],
                        [1, 0, 0],
                        [1, 1, 1]])
    img1[1:4, 1:4] = pattern
    
    # 入力画像2（パターンが右下にシフト）
    img2 = np.zeros((8, 8))
    img2[4:7, 4:7] = pattern
    
    # エッジ検出カーネル
    kernel = np.array([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]], dtype=float)
    
    # 畳み込み関数
    def convolve(img, kernel):
        h, w = img.shape
        kh, kw = kernel.shape
        out_h, out_w = h - kh + 1, w - kw + 1
        output = np.zeros((out_h, out_w))
        for i in range(out_h):
            for j in range(out_w):
                output[i, j] = np.sum(img[i:i+kh, j:j+kw] * kernel)
        return output
    
    # 畳み込み実行
    out1 = convolve(img1, kernel)
    out2 = convolve(img2, kernel)
    
    # 上段：画像1（元の位置）
    axes[0, 0].imshow(img1, cmap='Blues', vmin=0, vmax=1)
    axes[0, 0].set_title('入力1\n（パターンが左上）', fontsize=12)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(kernel, cmap='RdBu', vmin=-2, vmax=2)
    axes[0, 1].set_title('カーネル\n（Sobel横）', fontsize=12)
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(out1, cmap='RdBu')
    axes[0, 2].set_title('出力1\n（特徴が左上）', fontsize=12)
    axes[0, 2].axis('off')
    
    # 出力の最大値位置
    max_pos1 = np.unravel_index(np.abs(out1).argmax(), out1.shape)
    axes[0, 2].scatter([max_pos1[1]], [max_pos1[0]], c='red', s=100, marker='x')
    
    # 説明
    axes[0, 3].text(0.5, 0.5, f'最大応答位置:\n({max_pos1[0]}, {max_pos1[1]})', 
                   fontsize=14, ha='center', va='center',
                   bbox=dict(boxstyle='round', facecolor='lightyellow'))
    axes[0, 3].axis('off')
    
    # 下段：画像2（シフト後）
    axes[1, 0].imshow(img2, cmap='Blues', vmin=0, vmax=1)
    axes[1, 0].set_title('入力2\n（+3,+3シフト）', fontsize=12)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(kernel, cmap='RdBu', vmin=-2, vmax=2)
    axes[1, 1].set_title('同じカーネル\n（重み共有）', fontsize=12)
    axes[1, 1].axis('off')
    
    axes[1, 2].imshow(out2, cmap='RdBu')
    axes[1, 2].set_title('出力2\n（特徴も+3,+3シフト）', fontsize=12)
    axes[1, 2].axis('off')
    
    max_pos2 = np.unravel_index(np.abs(out2).argmax(), out2.shape)
    axes[1, 2].scatter([max_pos2[1]], [max_pos2[0]], c='red', s=100, marker='x')
    
    # 説明
    axes[1, 3].text(0.5, 0.6, f'最大応答位置:\n({max_pos2[0]}, {max_pos2[1]})', 
                   fontsize=14, ha='center', va='center',
                   bbox=dict(boxstyle='round', facecolor='lightyellow'))
    axes[1, 3].text(0.5, 0.3, f'シフト量:\n({max_pos2[0]-max_pos1[0]}, {max_pos2[1]-max_pos1[1]})', 
                   fontsize=14, ha='center', va='center',
                   bbox=dict(boxstyle='round', facecolor='lightgreen'))
    axes[1, 3].axis('off')
    
    plt.suptitle('平行移動等変性：入力のシフト → 出力も同じだけシフト', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"入力のシフト: (3, 3)")
    print(f"出力の最大応答位置の差: ({max_pos2[0]-max_pos1[0]}, {max_pos2[1]-max_pos1[1]})")
    print("→ 入力と同じだけ出力もシフト（平行移動等変性）")

demonstrate_translation_equivariance()

### なぜ重み共有が等変性をもたらすのか

1. **同じカーネル**がすべての位置で適用される
2. パターンが位置(a,b)にあっても位置(c,d)にあっても、**同じカーネル**で処理される
3. したがって、パターンが検出される**相対的な位置**は保存される

これは画像認識において非常に重要です：
- 猫が画像の左にいても右にいても「猫」として認識したい
- 重み共有により、位置に依存しないパターン検出が可能に

<a id="section5"></a>
## 5. 実装で確認

PyTorchで重み共有を確認してみましょう。

In [ ]:
def verify_weight_sharing_pytorch():
    """PyTorchで重み共有を確認"""
    print("="*70)
    print("PyTorchでの重み共有の確認")
    print("="*70)
    
    # 畳み込み層を作成
    conv = nn.Conv2d(1, 1, kernel_size=3, bias=False)
    
    # 重みを確認
    print(f"\n畳み込み層のパラメータ形状: {conv.weight.shape}")
    print(f"パラメータ数: {conv.weight.numel()}")
    
    # 入力画像（8x8）
    x = torch.zeros(1, 1, 8, 8)
    
    # 位置(1,1)と位置(4,4)に同じパターンを配置
    pattern = torch.tensor([[1., 2.], [3., 4.]])
    x[0, 0, 1:3, 1:3] = pattern
    x[0, 0, 4:6, 4:6] = pattern
    
    # 畳み込み実行
    with torch.no_grad():
        y = conv(x)
    
    print(f"\n入力の形状: {x.shape}")
    print(f"出力の形状: {y.shape}")
    
    # 同じパターンに対する応答を比較
    response_pos1 = y[0, 0, 0:2, 0:2].mean().item()
    response_pos2 = y[0, 0, 3:5, 3:5].mean().item()
    
    print(f"\n位置(1,1)付近の平均応答: {response_pos1:.4f}")
    print(f"位置(4,4)付近の平均応答: {response_pos2:.4f}")
    print(f"差: {abs(response_pos1 - response_pos2):.6f}")
    print("\n→ 同じパターンに対して同じ応答（重み共有の証拠）")
    
    return conv, x, y

conv, x, y = verify_weight_sharing_pytorch()

In [ ]:
def compare_conv_vs_fc_parameters():
    """畳み込みと全結合のパラメータ数を実際に比較"""
    print("="*70)
    print("PyTorchでのパラメータ数比較")
    print("="*70)
    
    # 設定
    input_size = 32
    input_channels = 3
    output_channels = 64
    kernel_size = 3
    
    # 畳み込み層
    conv = nn.Conv2d(input_channels, output_channels, kernel_size, padding=1)
    conv_params = sum(p.numel() for p in conv.parameters())
    
    # 全結合層（同じサイズの出力を得るため）
    fc_input = input_size * input_size * input_channels
    fc_output = input_size * input_size * output_channels
    fc = nn.Linear(fc_input, fc_output)
    fc_params = sum(p.numel() for p in fc.parameters())
    
    print(f"\n入力: {input_size}×{input_size}×{input_channels}")
    print(f"出力: {input_size}×{input_size}×{output_channels}")
    print(f"カーネルサイズ: {kernel_size}×{kernel_size}")
    
    print(f"\n【畳み込み層】")
    print(f"  重み: {conv.weight.shape}")
    print(f"  バイアス: {conv.bias.shape}")
    print(f"  合計パラメータ: {conv_params:,}")
    
    print(f"\n【全結合層】")
    print(f"  重み: {fc.weight.shape}")
    print(f"  バイアス: {fc.bias.shape}")
    print(f"  合計パラメータ: {fc_params:,}")
    
    print(f"\n【比較】")
    print(f"  削減率: {fc_params / conv_params:,.0f}倍")

compare_conv_vs_fc_parameters()

<a id="summary"></a>
## 6. まとめ

### 学んだこと

1. **重み共有の定義**
   - 同じカーネルを画像全体でスライドさせる
   - すべての位置で同じ重みを使用

2. **パラメータ効率**
   - 全結合層と比較して数千〜数十億倍のパラメータ削減
   - 局所的接続 + 重み共有の組み合わせ

3. **平行移動等変性**
   - 入力がシフト → 出力も同じだけシフト
   - 位置に依存しないパターン検出を実現

4. **なぜ画像に適しているか**
   - 同じパターン（エッジ、テクスチャ）は画像のどこにでも現れうる
   - 位置に関係なく同じ方法で処理すべき

In [ ]:
def summary_diagram():
    """まとめ図"""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # 中央: 重み共有
    from matplotlib.patches import FancyBboxPatch
    
    center_box = FancyBboxPatch((0.35, 0.4), 0.3, 0.2, 
                                boxstyle="round,pad=0.02",
                                facecolor='gold', edgecolor='orange', linewidth=2,
                                transform=ax.transAxes)
    ax.add_patch(center_box)
    ax.text(0.5, 0.5, '重み共有\n(Weight Sharing)', fontsize=16, ha='center', va='center',
           fontweight='bold', transform=ax.transAxes)
    
    # 上: 定義
    ax.text(0.5, 0.85, '定義: 同じカーネルを全位置で使用', fontsize=12, 
           ha='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightblue'))
    
    # 左: パラメータ効率
    ax.text(0.15, 0.5, 'パラメータ効率\n数千〜数十億倍削減', fontsize=11, 
           ha='center', va='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightgreen'))
    
    # 右: 平行移動等変性
    ax.text(0.85, 0.5, '平行移動等変性\n位置不変の検出', fontsize=11, 
           ha='center', va='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    # 下: 利点
    ax.text(0.5, 0.15, '画像認識への適合性\n「同じパターンは同じ方法で処理」', fontsize=12, 
           ha='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lavender'))
    
    # 矢印
    ax.annotate('', xy=(0.5, 0.65), xytext=(0.5, 0.8),
               arrowprops=dict(arrowstyle='->', color='blue', lw=2),
               transform=ax.transAxes)
    ax.annotate('', xy=(0.32, 0.5), xytext=(0.22, 0.5),
               arrowprops=dict(arrowstyle='<-', color='green', lw=2),
               transform=ax.transAxes)
    ax.annotate('', xy=(0.68, 0.5), xytext=(0.78, 0.5),
               arrowprops=dict(arrowstyle='<-', color='orange', lw=2),
               transform=ax.transAxes)
    ax.annotate('', xy=(0.5, 0.35), xytext=(0.5, 0.22),
               arrowprops=dict(arrowstyle='<-', color='purple', lw=2),
               transform=ax.transAxes)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('重み共有：CNNのパラメータ効率と等変性の源', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

summary_diagram()

### 次のノートブック

次のノートブックでは、**平行移動不変性と等変性**についてより詳しく学びます。

- 不変性（Invariance）と等変性（Equivariance）の違い
- CNNにおける等変性の正確な定義
- プーリングによる不変性の獲得